In [1]:
import os

import openmm as mm
from openmm.app import *
from openmm import unit
from openmm import app
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
import requests
import numpy as np
from math import sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
pdb = PDBFile('brs_full.pdb') #Sistema Vacio

In [3]:
forcefield = ForceField('amber14-all.xml', 'implicit/obc2.xml') #Se indica el solvente implicito

In [4]:
mm.GBSAOBCForce?

Init signature: mm.GBSAOBCForce(*args)
Docstring:     
This class implements an implicit solvation force using the GBSA-OBC model.


To use this class, create a GBSAOBCForce object, then call addParticle() once for each particle in the System to define its parameters. The number of particles for which you define GBSA parameters must be exactly equal to the number of particles in the System, or else an exception will be thrown when you try to create a Context. After a particle has been added, you can modify its force field parameters by calling setParticleParameters(). This will have no effect on Contexts that already exist unless you call updateParametersInContext().


When using this Force, the System should also include a NonbondedForce, and both objects must specify identical charges for all particles. Otherwise, the results will not be correct. Furthermore, if the nonbonded method is set to CutoffNonPeriodic or CutoffPeriodic, you should call setReactionFieldDielectric(1.0) on the 

In [5]:
gbsa = mm.GBSAOBCForce

In [6]:
gbsa.addParticle?

Signature: gbsa.addParticle(self, charge, radius, scalingFactor)
Docstring:
addParticle(self, charge, radius, scalingFactor) -> int
Add the GBSA parameters for a particle. This should be called once for each particle in the System. When it is called for the i'th time, it specifies the parameters for the i'th particle.

Parameters
----------
charge : double
    the charge of the particle, measured in units of the proton charge
radius : double
    the GBSA radius of the particle, measured in nm
scalingFactor : double
    the OBC scaling factor for the particle

Returns
-------
int
    the index of the particle that was added
File:      ~/MyOpt/Miniconda/envs/bifree2/lib/python3.7/site-packages/openmm/openmm.py
Type:      function


In [7]:
system_implicit = forcefield.createSystem(pdb.topology, soluteDielectric=1.0, solventDielectric=80.0, constraints=HBonds)

In [8]:
implicit = system_implicit.getForces()[5]
implicit

<openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7f33c16e2120> >

In [9]:
# Impresion de los nombres de los parametros que tiene el modelo de solvente implicito seleccionado

for ii in range(implicit.getNumPerParticleParameters()):
    print(implicit.getPerParticleParameterName(ii))

charge
or
sr


In [10]:
# Creacion de listas con los parametros involucrados en el modelo de solvente implicito seleccionado

charge  = []
or_     = []
sr      = []

for ii in range(implicit.getNumParticles()):
    aa, bb, cc = implicit.getParticleParameters(ii)
    charge.append(aa)
    or_.append(bb)
    sr.append(cc)

In [11]:
# Otra manera de crear listas con los parametros

parameters = [implicit.getParticleParameters(ii) for ii in range(implicit.getNumParticles())]
charge_2 = [parameters[ii][0] for ii in range(implicit.getNumParticles())]
or_2     = [parameters[ii][1] for ii in range(implicit.getNumParticles())]
sr_2     = [parameters[ii][2] for ii in range(implicit.getNumParticles())]

In [12]:
parameters_2 = [[implicit.getParticleParameters(ii)[jj] for jj in range(implicit.getNumPerParticleParameters())] for ii in range(implicit.getNumParticles())]

In [13]:
type(parameters[0])

tuple

In [14]:
type(parameters_2[0])

list

In [18]:
for ii in range(implicit.getNumParticles()):
    gbsa.addParticle(charge= charge[ii], radius= or_[ii], scalingFactor= sr[ii])

TypeError: addParticle() missing 1 required positional argument: 'self'

In [17]:
gbsa.addParticle(charge=0.1, radius=0.1, scalingFactor=0.1)

TypeError: addParticle() missing 1 required positional argument: 'self'